In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell, seq2seq
from tqdm import *
from tensorflow.python.ops import control_flow_ops
import sys
import random
import time

from Utils import Utils
import attention_cnn_lstm_lstm as model
from hyperboard import Agent

    
def train(train_utils, valid_utils, source_len, oseq_len, n_echos, simplified_len, decoder_hidden, encoder_hidden,
          embedding_size, batch_size, display_step, prob, lstm_layer, learning_rate,
          model_dir, source_nfilters, source_width):
            
    
    hyperparameters = {
        'learning rate': learning_rate,
        'batch size': batch_size,
        'criteria': '',
        'decoder hidden': decoder_hidden,
        'encoder_hidden':encoder_hidden,
        'prob': prob,
        'source_nfilters': source_nfilters,
        'source_width': source_width,
        'lstm_layer': lstm_layer,  
        'embedding_size': embedding_size
    }
        
    agent = Agent(port = 5001)
    
    hyperparameters['criteria'] = 'valid loss'
    name_valid_loss = agent.register(hyperparameters, 'cross entropy')
    
    hyperparameters['criteria'] = 'valid accu'
    name_valid_accu = agent.register(hyperparameters, 'accuracy')
    
    hyperparameters['criteria'] = 'train loss'
    name_train_loss = agent.register(hyperparameters, 'cross entropy')
    
    hyperparameters['criteria'] = 'train accu'
    name_train_accu = agent.register(hyperparameters, 'accuracy')
    
        
    if not os.path.isdir(model_dir):
        os.mkdir(model_dir)
        
    words_size = train_utils.get_words_size()
    
    outputs = model.build_model(words_size=words_size, 
                                embedding_size=embedding_size,
                                source_len=source_len,
                                simplified_len=simplified_len,
                                oseq_len=oseq_len, 
                                decoder_hidden=decoder_hidden,
                                encoder_hidden=encoder_hidden,
                                source_nfilters=source_nfilters,
                                source_width=source_width,
                                lstm_layer=lstm_layer, 
                                batch_size=batch_size, 
                                is_train=True)

    cost=outputs['cost']
    words_prediction=outputs['words_prediction']
    source=outputs['source']
    defendant=outputs['defendant']
    defendant_length=outputs['defendant_length']
    label=outputs['label']
    decoder_inputs=outputs['decoder_inputs']
    loss_weights=outputs['loss_weights']
    keep_prob=outputs['keep_prob']
    sample_rate=outputs['sample_rate']
    
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='adam_optimizer')
    # Compute the gradients for a list of variables.
    grads_and_vars = optimizer.compute_gradients(cost)
    # grads_and_vars is a list of tuples (gradient, variable).
    capped_grads_and_vars = [(tf.clip_by_norm(g, 5), v) for g,v in grads_and_vars]
    # Ask the optimizer to apply the capped gradients.
    train_op = optimizer.apply_gradients(capped_grads_and_vars)

    
    init = tf.initialize_all_variables()

    os.environ["CUDA_VISIBLE_DEVICES"] = '2'
    gpu_option = tf.GPUOptions(per_process_gpu_memory_fraction = 0.2)
    session_conf = tf.ConfigProto(allow_soft_placement = True, 
                                  log_device_placement = False,
                                  gpu_options = gpu_option)
    
    sess = tf.Session()
    
#     pre_saver = tf.train.Saver()
#     ckpt = tf.train.latest_checkpoint('/home/xuwenshen/2017_3_13/attention_cnn_lstm-lstm/model_v1/')
#     pre_saver.restore(sess, ckpt)
    
    global_steps = 0
    
    echos = 0
    batch = 0
    
    saver = tf.train.Saver(max_to_keep = 20)

    sess.run(init)
    
    tvar = tf.trainable_variables()
    for v in tvar:
        print (v.name)
    
    print ('init done')
    
    train_cost = 0
    train_accu = 0
    train_bleu = 0
    
    while True:
        
        datas, is_again = train_utils.next_batch()
        
        if is_again:
            
            train_cost = 0
            train_accu = 0
            echos += 1
            batch = 0
            if echos == n_echos:
                break
            continue
            
        
        batch_source = datas['source'] 
        batch_defendant = datas['defendant'] 
        batch_defendant_length = datas['defendant_length']
        batch_ground_truth = datas['ground_truth']
        batch_label = datas['label']
        batch_weights = datas['loss_weights']
        
        batch += 1
        

        sample_rate_ = min(.5, 0.01 * (global_steps))

        feed_dic = {source:batch_source,
                    defendant:batch_defendant,
                    label:batch_label,
                    defendant_length:batch_defendant_length,
                    decoder_inputs:batch_ground_truth,
                    loss_weights:batch_weights,
                    keep_prob:prob,
                    sample_rate:sample_rate_}
        
        words_, cost_, _= sess.run([words_prediction, cost, train_op], feed_dict= feed_dic)
#         accu_, bleu_ = train_utils.i2t(words_, to_print = False)
    
        train_cost += cost_
#         train_accu += accu_
#         train_bleu += bleu_
        

        
        if batch % display_step == 0:
            
            valid_cost = 0
            valid_accu = 0
            valid_bleu = 0
            valid_batchs = 0
            
            train_cost /= display_step
            train_accu /= display_step
            train_bleu /= display_step
            
            while True:
                
                datas, is_again = valid_utils.next_batch()
                
                if is_again:
                    break
        
                batch_source = datas['source'] 
                batch_defendant = datas['defendant'] 
                batch_defendant_length = datas['defendant_length']
                batch_ground_truth = datas['ground_truth']
                batch_label = datas['label']
                batch_weights = datas['loss_weights']

                valid_batchs += 1
                
                feed_dic = {source:batch_source,
                            defendant:batch_defendant,
                            label:batch_label,
                            defendant_length:batch_defendant_length,
                            decoder_inputs:batch_ground_truth,
                            loss_weights:batch_weights,
                            keep_prob:1.,
                            sample_rate:1.}
                    
                word_, cost_, = sess.run([words_prediction, cost], feed_dict=feed_dic)     
                accu_, bleu_ = valid_utils.i2t(word_, to_print = True)
                
                valid_cost += cost_
                valid_accu += accu_
                valid_bleu += bleu_
            
            valid_cost /= valid_batchs
            valid_accu /= valid_batchs
            valid_bleu /= valid_batchs
            
            saver.save(sess, model_dir + \
                       "sample_rate-" + str(sample_rate_) + \
                       "-train_accu-{:.3f}".format(train_accu) + \
                       "-train_cost-{:.3f}".format(train_cost) + \
                       "-train_bleu-{:.3f}".format(train_bleu) + \
                       "-valid_accu-{:.3f}".format(valid_accu) + \
                       "-valid_cost-{:.3f}".format(valid_cost) + \
                       "-valid_bleu-{:.3f}".format(valid_bleu) + \
                       "-model.ckpt", global_step = global_steps)
            
            
            print ("Echo: "+str(echos) + " Iters: "+str(batch) + \
                   " Sample: " + "{:.3f}".format(sample_rate_) + \
                   " Train loss: " + "{:.3f}".format(train_cost) + \
                   " Valid loss: " + "{:.3f}".format(valid_cost) + \
                   " Train accu: " + "{:.3f}".format(train_accu) + \
                   " Valid accu: " + "{:.3f}".format(valid_accu) + \
                   " Train bleu: " + "{:.3f}".format(train_bleu) + \
                   " Valid bleu: " + "{:.3f}".format(valid_bleu))
            

            agent.append(name_valid_loss, global_steps, valid_cost)
            agent.append(name_valid_accu, global_steps, valid_accu)
            agent.append(name_train_loss, global_steps, train_cost)
            agent.append(name_train_accu, global_steps, train_accu)
            
            global_steps += 1
            
        
            
if __name__ == '__main__':
    
    words_path = '/home/xuwenshen/data/big_data/2017_3_13/words'
    train_path = '/home/xuwenshen/data/big_data/2017_3_13/train.h5'
    valid_path = '/home/xuwenshen/data/big_data/2017_3_13/valid.h5'
    
    oseq_len = 200
    source_len = 1000
    simplified_len = 150

    batch_size = 50
    display_step = 400
    n_echos = 100
    prob = 0.75
    decoder_hidden = 700
    encoder_hidden = 312
    source_nfilters = 450
    source_width = 3
    lstm_layer = 1
    learning_rate = 0.001
    embedding_size = 200


    
    
    train_utils = Utils(words_path, train_path, batch_size, nb_samples=1600000)
    valid_utils = Utils(words_path, valid_path, batch_size, nb_samples=640)

    train(train_utils=train_utils,
          valid_utils=valid_utils,
          source_len=source_len,
          simplified_len=simplified_len,
          oseq_len=oseq_len,
          embedding_size=embedding_size,
          batch_size=batch_size,
          display_step=display_step,
          n_echos=n_echos,
          prob=prob,
          decoder_hidden=decoder_hidden,
          encoder_hidden=encoder_hidden,
          source_nfilters=source_nfilters,
          source_width=source_width,
          lstm_layer=lstm_layer,
          learning_rate=learning_rate,
          model_dir='/home/xuwenshen/2017_3_13/attention_cnn_lstm-lstm/model_v1/')
    
    
    

[None, 998, 1, 450]
[None, 996, 1, 450]
[None, 994, 1, 450]
build model 
embedding:0
conv_source_w_1:0
conv_source_w_2:0
conv_source_w_3:0
conv_source_b_1:0
conv_source_b_2:0
conv_source_b_3:0
conv_source_scale_1:0
conv_source_scale_2:0
conv_source_scale_3:0
conv_source_offset_1:0
conv_source_offset_2:0
conv_source_offset_3:0
generation_w:0
generation_b:0
dynamic_rnn_encoder/BasicLSTMCell/Linear/Matrix:0
dynamic_rnn_encoder/BasicLSTMCell/Linear/Bias:0
rnn_decoder/AttnW_0:0
rnn_decoder/AttnV_0:0
rnn_decoder/Linear/Matrix:0
rnn_decoder/Linear/Bias:0
rnn_decoder/BasicLSTMCell/Linear/Matrix:0
rnn_decoder/BasicLSTMCell/Linear/Bias:0
rnn_decoder/Attention_0/Linear/Matrix:0
rnn_decoder/Attention_0/Linear/Bias:0
rnn_decoder/AttnOutputProjection/Linear/Matrix:0
rnn_decoder/AttnOutputProjection/Linear/Bias:0
init done
罪犯牟成林，男，1988年1月6日出生，汉族，山东省青州市人，初中文化，原住山东省青州市何官镇南小王村朝阳街56号。现在云南省第三监狱服刑。云南省德宏傣族颇族自治州中级人民法院于二○○六年十一月二日以（2006）德刑初字第1201号刑事判决，以被告人牟成林犯运输毒品罪，判处有期徒刑十五年。判决发生法律效力后，交付执行。执行期间，因确有悔改表现，于二○○九年五

/home/xuwenshen/py3/lib/python3.4/site-packages/nltk/translate/bleu_score.py:472: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


原公诉机关广东省东莞市第一市区人民检察院。上诉人（原审被告人）黄平，男。因涉嫌犯贩卖毒品罪于2013年7月10日被羁押，同月11日被刑事拘留，同年8月17日被逮捕。现押于东莞市看守所（上桥）。广东省东莞市第一人民法院审理广东省东莞市第一市区人民检察院指控原审被告人黄平犯贩卖毒品罪一案，于2014年1月22日作出（2013）东一法刑初字第1844号刑事判决：一、被告人黄平犯贩卖毒品罪，判处有期徒刑九年，并处罚金人民币5000元。二、随案移送的手机二部、电子一台及人民币2190元，予以没收，上缴国库。宣判后，被告人黄平不服，提出上诉。本院审理过程中，上诉人黄平申请撤回上诉。
--------------------

原审被告人，因涉嫌犯贩卖毒品罪于2013年7月10日被羁押，被告人黄平不服，广东省东莞市第一人民法院审理广东省东莞市第一市区人民检察院指控原审被告人黄平犯贩卖毒品罪一案，东一法刑初字第1844号刑事判决：一、被告人黄平犯贩卖毒品罪，
--------------------

本院认为，上诉人黄平撤回上诉的意思表示真实，且原判认定事实和适用法律正确，量刑适当
--------------------

本院认为，原判认定事实不清

********************

罪犯牟成林，男，1988年1月6日出生，汉族，山东省青州市人，初中文化，原住山东省青州市何官镇南小王村朝阳街56号。现在云南省第三监狱服刑。云南省德宏傣族颇族自治州中级人民法院于二○○六年十一月二日以（2006）德刑初字第1201号刑事判决，以被告人牟成林犯运输毒品罪，判处有期徒刑十五年。判决发生法律效力后，交付执行。执行期间，因确有悔改表现，于二○○九年五月十日、二○一○年十一月七日、二○一二年五月十日、二○一三年五月十日经本院四次裁定共减刑五年。执行机关云南省第三监狱于二○一四年五月十一日提出假释建议书，报送本院审理。本院受理后依法公示，公示期间没有异议，经组成合议庭进行审理，现已审理终结。执行机关以罪犯牟成林在刑罚执行期间，认真遵守监规，接受教育改造，确有悔改表现为由，建议对该犯假释。经审理查明，罪犯牟成林在刑罚执行期间，认罪悔罪；认真遵守法律法规及监规，接受教育改造；积极参加思想、文化、职业技术教育；积极参加劳动，努力完成各项劳动任务，获记累计分考核表扬二次。上述事实，有执行机